This notebook generates binaural sound examples for the common slopes amplitudes interpolation problem. 
First the soundfield at octave bands is generated by getting the learned amplitudes from the DNN in octave bands. Then an ambisonics RIR is
reconstructed from the learned amplitudes using white noise shaping.

Simultaneously, an HRTF dataset is loaded and converted to the ambisonics domain. The ambisonics RIRs are first rotated, according to the head orientation and then convolved with the HRTFs' SH representation. This rotated soundfield is then convolved with the input mono signal to get the binauralised output.

In [ ]:
import torch
import numpy as np
from pathlib import Path
import os

from spatial_sampling.inference import get_ambisonic_rirs
from spatial_sampling.dataloader import parse_room_data
from src.sofa_parser import HRIRSOFAReader

### Create a trajectory of a listener moving across the space

In [ ]:
# along x axis between three rooms
start_pos_x, start_pos_y = (0.5, 3.5)
end_pos_x, end_pos_y = (9, 3.5)
num_pos = 50

linear_trajectory_x = np.linspace(start_pos_x, end_pos_x, num_pos)
linear_trajectory_y = np.linspace(start_pos_y, end_pos_y, num_pos)
linear_trajectory_z = 1.5 * np.ones(num_pos)

rec_pos_list = np.zeros((num_pos, 3))
rec_pos_list[:, 0] = linear_trajectory_x
rec_pos_list[:, 1] = linear_trajectory_y
rec_pos_list[:, 2] = linear_trajectory_z

# along y-axis between rooms 2 and 3
start_pos_x, start_pos_y = (9.1, 3.5)
end_pos_x, end_pos_y = (9.0, 12.0)
num_pos = 68

linear_trajectory_x = np.linspace(start_pos_x, end_pos_x, num_pos)
linear_trajectory_y = np.linspace(start_pos_y, end_pos_y, num_pos)
linear_trajectory_z = 1.5 * np.ones(num_pos)

rec_pos_list = np.vstack((rec_pos_list, np.vstack((linear_trajectory_x, linear_trajectory_y, linear_trajectory_z)).T))


### Get the true and predicted room datasets with its corresponding ambisonics RIR

In [ ]:
room_data_pkl_path = Path('resources/Georg_3room_FDTD/srirs_spatial.pkl').resolve()
output_pkl_path = Path('output/spatial_sampling/grid_rir_treble_ambi_rirs.pkl').resolve()
config_path = Path('data/config/spatial_sampling/').resolve()

# get the full band dataset
true_cs_room_data = parse_room_data(room_data_pkl_path)

In [ ]:
from importlib import reload
import spatial_sampling
reload(spatial_sampling.inference)
from spatial_sampling.inference import get_ambisonic_rirs

# get predicted output from the trained models
if not os.path.exists(output_pkl_path):
    pred_cs_room_data = get_ambisonic_rirs(rec_pos_list, output_pkl_path, true_cs_room_data, config_path)
else:
    with open(output_pkl_path, "rb") as f:
        pred_cs_room_data = pickle.load(f)

### Load the HRTF dataset

In [ ]:
hrtf_path = Path('resources/HRTF/48kHz/KEMAR_Knowl_EarSim_SmallEars_FreeFieldComp_48kHz.sofa')
hrtf_reader = HRIRSOFAReader(hrtf_path, true_cs_room_data.ambi_order)
